In [9]:
# %pip install librosa matplotlib

In [1]:
import os
import numpy as np
import librosa
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
import IPython.display
import numpy as np
from PIL import Image
from rembg import remove
import cv2

# from concurrent.futures import ProcessPoolExecutor
# import multiprocessing

### 데이터증강 기법
- Contrast
- Filp (뒤집기)
- Rotate (0, 45, 90, 135, 180, 225, 270, 315)
- Blur (Median), 커널사이즈 7
- Noise

In [2]:
def augment_image(image_path, output_dir):
    original_image = Image.open(image_path)
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    
    for angle in [0, 45, 90, 135, 180, 225, 270, 315]:
        rotated_image = original_image.rotate(angle)
        
        # Convert PIL Image to OpenCV format
        cv_image = cv2.cvtColor(np.array(rotated_image), cv2.COLOR_RGB2BGR)
        
        # Contrast
        contrast_img = cv2.convertScaleAbs(cv_image, alpha=3.0, beta=10)
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}_contrast.jpg"), contrast_img)
        
        # Flip
        flipped_image = cv2.flip(cv_image, 1)  # 1 for horizontal flip
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}_flipped.jpg"), flipped_image)
        
        # Median Blur
        blurred_image = cv2.medianBlur(cv_image, 7)  # 5 is the kernel size
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}_blurred.jpg"), blurred_image)
        
        # Noise
        noise = np.random.normal(0, 25, cv_image.shape).astype(np.uint8)
        noisy_image = cv2.add(cv_image, noise)
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}_noisy.jpg"), noisy_image)
        
        # Save the original rotated image
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}.jpg"), cv_image)

def augment_all_images(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir, filename)
            augment_image(input_path, output_dir)

# 사용 예시
augment_all_images('/home/data/train', '/home/data/augmented(contrast,flip,blur,noise)')

### 배경제거

In [2]:
from concurrent.futures import ThreadPoolExecutor
import os
from PIL import Image
from rembg import remove

def remove_img_bg(input_path, output_path):
    with Image.open(input_path) as input_image:
        output_image = remove(input_image)
        output_image.save(output_path)

def process_image(filename, input_dir, output_dir):
    input_path = os.path.join(input_dir, filename)
    output_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_no_bg.png")
    remove_img_bg(input_path, output_path)
    print(f"Processed: {filename}")

def remove_all_images(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    with ThreadPoolExecutor() as executor:
        executor.map(lambda f: process_image(f, input_dir, output_dir), image_files)

# 사용 예시
remove_all_images('/home/data/train', '/home/data/augmented_rembg')

Processed: 706cc6881b18b291.jpg
Processed: 65453fe7b270cd4a.jpg
Processed: 15f0ed8589380d8f.jpg
Processed: bdc0e937c23d0544.jpg
Processed: 6d6ffab07161b8be.jpg
Processed: cee86faf6225599e.jpg
Processed: 8c04b33be2485613.jpg
Processed: 5f489ecf342e00d2.jpg
Processed: 8ea17c5687d735b3.jpg
Processed: 4826a34b61ae4e98.jpg
Processed: fba49012c202152d.jpg
Processed: 60736fb522fa9311.jpg
Processed: c323c86de8033ad1.jpg
Processed: 572a127d4afbe039.jpg
Processed: fc85f507537c9300.jpg
Processed: a4665a740c6acbe9.jpg
Processed: a0b9e86061135d91.jpg
Processed: 82b19ea64a4ea2dc.jpg
Processed: 3e9932b25b0373a2.jpg
Processed: 822f9d89d790a354.jpg
Processed: d941da0b32375b7b.jpg
Processed: b993227004156755.jpg
Processed: a0d422140ba03bcb.jpg
Processed: 9b3dbfeb10499aa7.jpg
Processed: 5af840cc6f074b98.jpg
Processed: a7b4e66d98cde556.jpg
Processed: 8ecdba0e9dab2487.jpg
Processed: 3d81b423a1d5c984.jpg
Processed: 5a434571d74cf4a5.jpg
Processed: 8b3b6ef2484a4535.jpg
Processed: be3aed2be00fd713.jpg
Processe

KeyboardInterrupt: 

### 학습용 Target CSV 만들기

In [4]:
import csv
import os

def update_csv(original_csv_path, augmented_image_dir, output_csv_path):
    # 원본 CSV 파일 읽기
    with open(original_csv_path, 'r') as f:
        reader = csv.DictReader(f)
        original_data = list(reader)

    # 새로운 데이터를 저장할 리스트
    new_data = []

    # 증강된 이미지에 대한 데이터 추가
    for row in original_data:
        original_filename = row['ID']
        target = row['target']
        
        # 파일 이름에서 확장자 제거
        base_name = os.path.splitext(original_filename)[0]
        
        for angle in [0, 45, 90, 135, 180, 225, 270, 315]:
            # Contrast 이미지
            contrast_filename = f"{base_name}_rotated_{angle}_contrast.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, contrast_filename)):
                new_data.append({
                    'ID': contrast_filename,
                    'target': target
                })
            
            # 회전된 이미지
            rotated_filename = f"{base_name}_rotated_{angle}.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, rotated_filename)):
                new_data.append({
                    'ID': rotated_filename,
                    'target': target
                })
            
            # Flipped 이미지
            flipped_filename = f"{base_name}_rotated_{angle}_flipped.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, flipped_filename)):
                new_data.append({
                    'ID': flipped_filename,
                    'target': target
                })
            
            # Blurred 이미지
            blurred_filename = f"{base_name}_rotated_{angle}_blurred.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, blurred_filename)):
                new_data.append({
                    'ID': blurred_filename,
                    'target': target
                })
            
            # Noisy 이미지
            noisy_filename = f"{base_name}_rotated_{angle}_noisy.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, noisy_filename)):
                new_data.append({
                    'ID': noisy_filename,
                    'target': target
                })

    # 새로운 CSV 파일 작성
    with open(output_csv_path, 'w', newline='') as f:
        fieldnames = ['ID', 'target']
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        
        writer.writeheader()
        for row in new_data:
            writer.writerow(row)

    print(f"Updated CSV file with only augmented data has been saved to {output_csv_path}")

# 사용 예시
update_csv('/home/train_csv/train(label_change).csv', '/home/data/augmented(contrast,flip,blur,noise)', '/home/train_csv/train(contrast, flip, blur, noise).csv')

Updated CSV file with only augmented data has been saved to /home/train_csv/train(contrast, flip, blur, noise).csv


In [1]:
# 두 폴더(/home/data/train, /home/data/augmented_train)의 이미지 파일 합쳐서 하나의 새로운 폴더(/home/data/rotate_train)로 이동
import os
import shutil

def move_images(input_dir1, input_dir2, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(input_dir1):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir1, filename)
            output_path = os.path.join(output_dir, filename)
            shutil.copy(input_path, output_path)
    
    for filename in os.listdir(input_dir2):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir2, filename)
            output_path = os.path.join(output_dir, filename)
            shutil.copy(input_path, output_path)
            
move_images('/home/data/train', '/home/data/augmented_train', '/home/train_csv/rotate_train')

In [4]:
train_path = "/home/data/augmented_train"
cnt = 0
for root, dirs, files in os.walk(train_path):
    cnt += len(files)
    
print(f"Total number of files in {train_path}: {cnt}")

Total number of files in /home/data/augmented_train: 25120


In [8]:
import pandas as pd

data = "/home/data/train_augmented.csv"
data = pd.read_csv(data)
data.head(20)

,ID,target
0,002f99746285dfdd_rotated_0.jpg,16
1,002f99746285dfdd_rotated_0_flipped.jpg,16
2,002f99746285dfdd_rotated_0_blurred.jpg,16
3,002f99746285dfdd_rotated_0_noisy.jpg,16
4,002f99746285dfdd_rotated_90.jpg,16
5,002f99746285dfdd_rotated_90_flipped.jpg,16
6,002f99746285dfdd_rotated_90_blurred.jpg,16
7,002f99746285dfdd_rotated_90_noisy.jpg,16
8,002f99746285dfdd_rotated_180.jpg,16
9,002f99746285dfdd_rotated_180_flipped.jpg,16
